### Homework: Agents

In this homework, we will learn more about function calling, and we will also explore MCP - model-contect-protocol.

In [1]:
import random

### Preparation

First, we'll define a function that we will use when building our agent.

It will generate fake weather data:

In [2]:
known_weather_data = {
    'berlin': 20.0
}

In [3]:
def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description

We want to use it as a tool for our agent, so we will need to describe it.

In [4]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather data",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The question about weather for a location"
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

What did you put in TODO3?

### A1. TODO3 has the value "query"

### Testing (Optional)

If you have an OpenAI API Key (or alternative provider), let's test it.

A question could be "What's the weather like in Germany?"

Experiment with different system prompts to have better answers from the system.

You can use [chat_assistant.py](https://github.com/alexeygrigorev/rag-agents-workshop/blob/main/chat_assistant.py)
or implement everything yourself 

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
```

In [5]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'The question about weather for a location'}},
   'required': ['query'],
   'additionalProperties': False}}]

### Q2. Adding Another Tool

Let's add another tool - a function that can add weather data to our database:

In [6]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Now let's write a description for it.

### A2. Description for set_weather

In [7]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Adds weather data to the database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Gets the weather for a location"
            },
            "weather_location": {
                "type": "string",
                "description": "Name of city",
            },
            "temperature": {
                "type": "float",
                "description": "Temperature"
            }
        },
        "required": ["query", "weather_location", "temperature"],
        "additionalProperties": False
    }
}

In [8]:
tools.add_tool(set_weather, set_weather_tool)

tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'The question about weather for a location'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Adds weather data to the database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Gets the weather for a location'},
    'weather_location': {'type': 'string', 'description': 'Name of city'},
    'temperature': {'type': 'float', 'description': 'Temperature'}},
   'required': ['query', 'weather_location', 'temperature'],
   'additionalProperties': False}}]

### MCP

MCP stands for Model-Contect Protocol.  It allows LLMs to communicate with different tools (like Qdrant).  It's function calling, but one step further:

* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

### Q3. Install FastMCP

Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):

In [9]:
%pip install fastmcp

Note: you may need to restart the kernel to use updated packages.


What is the version of FastMCP you installed?

In [10]:
import fastmcp
print(fastmcp.__version__)

2.10.4


### A3. We are using FastMCP version 2.10.4

### Q4. Simple MCP Server

A simple MCP server from the documentation looks like:

```python
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()
```

In our case, we need to write docstrings for our functions.

Implemented in weather_server.py

```python
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

if __name__ == "__main__":
    mcp.run()

```

What do you see in the output?

### A4. The output shows Starting MCP server 'Demo 🚀' with transport STDIO

### Q5. Protocol

There are different ways to communicate with an MCP server.  Ours is currently running using standard input/output, which means the client writes something to stdin and reads the answer using stdout.

Our weather server is currently running.

```bash
fastmcp run weather_server.py:mcp
```

This is how we start communicating with it:

First, we send an initialization request -- this way, we register our client with the server:

In [11]:
{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2024-11-05",
        "capabilities": {
            "roots": {
                "listChanged": True
            },
            "sampling": {}
        },
        "clientInfo": {
            "name": "test-client",
            "version": "1.0.0"
        }
    }
}


{'jsonrpc': '2.0',
 'id': 1,
 'method': 'initialize',
 'params': {'protocolVersion': '2024-11-05',
  'capabilities': {'roots': {'listChanged': True}, 'sampling': {}},
  'clientInfo': {'name': 'test-client', 'version': '1.0.0'}}}

We should get back something like that, which is an acknowledgement of the request:

 Next, we reply back, confirming the initialization:

In [12]:
{"jsonrpc": "2.0", "method": "notifications/initialized"}

{'jsonrpc': '2.0', 'method': 'notifications/initialized'}

Now we can ask for a list of available methods:

In [50]:
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}

{'jsonrpc': '2.0', 'id': 2, 'method': 'tools/list'}

We don't expect to get anything in response

In [13]:
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "set_weather", "arguments": {"city": "berlin", "temp": 20.0}}}

{'jsonrpc': '2.0',
 'id': 3,
 'method': 'tools/call',
 'params': {'name': 'set_weather',
  'arguments': {'city': 'berlin', 'temp': 20.0}}}

Let's ask the temperature in Berlin:

In [14]:
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "berlin"}}}

{'jsonrpc': '2.0',
 'id': 3,
 'method': 'tools/call',
 'params': {'name': 'get_weather', 'arguments': {'city': 'berlin'}}}

What did you get in response?

### A5. Received a response of:

```
{'jsonrpc': '2.0',
 'id': 3,
 'method': 'tools/call',
 'params': {'name': 'get_weather', 'arguments': {'city': 'berlin'}}}
```

### Q6. Client

We typically do not interact with the server by copy-pasting commands in the terminal.

In practice, we use an MCP client.  Let's implement it.

FastMCP also supports MCP clients:

```python
async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```

Use the client to get the list of available tools in our script.  How does the result loook?

If you are running the code in Jupyter, you need to pass an instance of the MCP server to the `Client `:

```python
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....
```

If you run it as a script, you will need to use asyncio:

```python
import asyncio

async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())
```

Copy the output with the available tools when filling in the homework form.

Note: Jupyter already runs an async event loop, you should NOT use asyncio.run(). Just use await directly:

In [26]:
#import asyncio
from fastmcp import Client
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(f"Available tools: {tools}")
        #forecast = await mcp_client.call_tool("get_weather", {"city": "Berlin"})
        #print(f"🌤 Forecast in Berlin: {forecast.output}")

if __name__ == "__main__":
    await main()

Available tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem

### Using tools from the MCP server (optional)

FastMCP uses asyncio for client-server communication.  In our case, the we wrote previously in the module (chat_assistant.py) is not asyncio-friendly, so it will require a lot of adjustments to run it.

Which is why we asked Claude to implement a simple non-async MCP client (see [mcp_client.py](mcp_client.py)) that can only do this:

- List tools
- Invoke the specified tool

Note: this is not a production-ready MCP Client!  Use it only for learning purposes.

Check the code - it's quite illustrative.  Or experiment with writing this code yourself.

Here's how we can use it:

In [57]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.10.1'}}
Sending initialized notification...
Handshake completed successfully


While it's somewhat verbose, it follows the initialization structure we outlined in Q5.

Now we can use it:

In [58]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '-0.9'}],
 'structuredContent': {'result': -0.9},
 'isError': False}

In order to include it in our existing application, we need a wrapper class:

In [59]:
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

It's very similar to the `Tools` class we created in the module, but it uses MCP to communicate with the MCP server.

(Where `convert_tools_list` converts MCP functions description format into the OpenAI's one)

Let's use it:

In [ ]:
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=our_mcp_client 
)

chat.run()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.10.1'}}
Sending initialized notification...
Handshake completed successfully


Now we can use the MCP server for function calling!